In [1]:
import pickle
import numpy as np

from utils.funcs import *
from tqdm.autonotebook import tqdm

/home/ytian/anaconda3/envs/receptorgpt/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
with open("data/tcr_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)
len(embeddings)

34099

In [3]:
len(embeddings[0])

320

In [4]:
tcrs = pd.read_parquet("data/tcrs_final.parquet")
tcrs.head()

,Species,Antigen Epitope,Antigen Protein,Antigen Source,CDR3.beta.aa,TRBV,TRBJ,Reference,Database
0,Human,KLGGALQAK,IE1,CMV,CASSPKTSVTYNEQFF,TRBV7-9*01,TRBJ2-1*01,https://www.10xgenomics.com/resources/applicat...,VDJdb
1,Human,NLVPMVATV,pp65,CMV,CASSLDSLNTIYF,TRBV5-1*01,TRBJ1-3*01,https://pubmed.ncbi.nlm.nih.gov/28423320,VDJdb
2,Human,KLGGALQAK,IE1,CMV,CASSSRTSSTDTQYF,TRBV12-4*01,TRBJ2-3*01,https://www.10xgenomics.com/resources/applicat...,VDJdb
3,Human,KLGGALQAK,IE1,CMV,CSSESGTSEAFF,TRBV29-1*01,TRBJ1-1*01,https://www.10xgenomics.com/resources/applicat...,VDJdb
4,Human,KLGGALQAK,IE1,CMV,CSVEYGLAGSTDTQYF,TRBV29-1*01,TRBJ2-3*01,https://www.10xgenomics.com/resources/applicat...,VDJdb


In [5]:
print(len(embeddings), len(tcrs))

34099 34099


In [6]:
search_embedding(embeddings[0], 1, tcrs.loc[0, "Species"])

{'ids': ['3e4da022-1f56-4f8f-98d7-6f076fb939c2'],
 'metadatas': [{'Antigen Epitope': 'KLGGALQAK',
   'Antigen Protein': 'IE1',
   'Antigen Source': 'CMV',
   'CDR3.beta.aa': 'CASSPKTSVTYNEQFF',
   'Database': 'VDJdb',
   'Reference': 'https://www.10xgenomics.com/resources/application-notes/a-new-way-of-exploring-immunity-linking-highly-multiplexed-antigen-recognition-to-immune-repertoire-and-phenotype/#',
   'Species': 'Human',
   'TRBJ': 'TRBJ2-1*01',
   'TRBV': 'TRBV7-9*01'}],
 'scores': [0.0]}

In [7]:
def top_match_cdr3(embedding, species):
    try:
        result = search_embedding(embedding, 1, species)
        pred_cdr3 = result["metadatas"][0]["CDR3.beta.aa"]
    except:
        pred_cdr3 = np.nan
    return pred_cdr3

In [8]:
from joblib import Parallel, delayed

In [9]:
# run on all embeddings in parallel
pred_cdr3s = Parallel(n_jobs=-1)(
    delayed(top_match_cdr3)(embedding, tcrs.loc[i, "Species"])
    for i, embedding in tqdm(enumerate(embeddings), total=len(embeddings))
)

100%|██████████| 34099/34099 [07:07<00:00, 79.69it/s]


In [13]:
true_cdr3s = tcrs["CDR3.beta.aa"].tolist()

In [14]:
print(len(pred_cdr3s), len(true_cdr3s))

34099 34099


In [15]:
accuracy = (np.array(true_cdr3s) == np.array(pred_cdr3s)).mean()
print(f"Accuracy: {accuracy*100:.1f}%")

Accuracy: 93.7%
